In [91]:
import torch

from liptrf.models.moderate import CIFAR10_4C3F_ReLUx

In [96]:
weights = torch.load('../weights/CIFAR10_4c3f_relux_seed-2_pass1.pt')
post_weights = torch.load('../weights/CIFAR10_4c3f_relux_seed-2_pass1_lc_alpha-0.01_eta-0.1_lc_gamma-0.98_lr-1.2_checkpoint.pt')#['weights']

In [97]:
for k in weights.keys():
    if 'weight' in k:
        print (k, torch.nonzero(weights[k].flatten()).shape, torch.nonzero(post_weights[k].flatten()).shape, torch.linalg.norm(weights[k]).item(), torch.linalg.norm(post_weights[k]).item())

conv1.weight torch.Size([87, 1]) torch.Size([18, 1]) 4.440517902374268 2.535034656524658
conv2.weight torch.Size([1639, 1]) torch.Size([291, 1]) 7.439726829528809 4.345742225646973
conv3.weight torch.Size([1844, 1]) torch.Size([363, 1]) 8.585911750793457 5.387950420379639
conv4.weight torch.Size([6554, 1]) torch.Size([1224, 1]) 13.510056495666504 10.100250244140625
fc1.weight torch.Size([209715, 1]) torch.Size([26134, 1]) 74.03863525390625 41.417545318603516
fc2.weight torch.Size([26215, 1]) torch.Size([3170, 1]) 27.979583740234375 12.1904935836792
fc3.weight torch.Size([512, 1]) torch.Size([92, 1]) 9.992071151733398 5.977871417999268
